# libraries

In [1]:
import os
from google.cloud import bigquery
import pandas as pd

# Absolute path to your key
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/tolgasabanoglu/Desktop/github/spatiotemporal/spatiotemporal-key.json"

# Initialize BigQuery client
client = bigquery.Client()


In [10]:
table_id = f"{client.project}.garmin_data.garmin_raw_data"
df = client.query(f"SELECT * FROM `{table_id}`").to_dataframe()
df

/Users/tolgasabanoglu/Desktop/github/spatiotemporal/spatiotemporal_env/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,filename,raw_json
0,body_battery_2025-09-22.json,"[{""date"": ""2025-09-22"", ""charged"": 45, ""draine..."
1,heart_rate_2025-04-17.json,"{""userProfilePK"": 130724645, ""calendarDate"": ""..."
2,body_battery_2025-05-01.json,"[{""date"": ""2025-05-01"", ""charged"": 36, ""draine..."
3,sleep_2025-04-15.json,"{""dailySleepDTO"": {""id"": 1744676720000, ""userP..."
4,sleep_2025-06-28.json,"{""dailySleepDTO"": {""id"": 1751058151000, ""userP..."
...,...,...
3397,steps_2025-09-19.json,"[{""startGMT"": ""2025-09-18T15:00:00.0"", ""endGMT..."
3398,steps_2025-07-07.json,"[{""startGMT"": ""2025-07-06T22:00:00.0"", ""endGMT..."
3399,stress_2025-09-12.json,"{""userProfilePK"": 130724645, ""calendarDate"": ""..."
3400,stress_2025-06-21.json,"{""userProfilePK"": 130724645, ""calendarDate"": ""..."


In [11]:
import json
import pandas as pd

# Identify the type of JSON structure
def detect_json_type(raw):
    try:
        obj = json.loads(raw)
        if isinstance(obj, list):
            return 'list'
        elif isinstance(obj, dict):
            return 'dict'
        else:
            return 'other'
    except Exception:
        return 'invalid'

df['json_type'] = df['raw_json'].apply(detect_json_type)

# See how many of each type we have
print(df['json_type'].value_counts())

# Peek at unique filename prefixes (the metric types)
df['metric'] = df['filename'].str.extract(r'^([^_]+)')
print(df['metric'].value_counts())


json_type
dict    2043
list    1359
Name: count, dtype: int64
metric
body      681
heart     681
sleep     681
stress    681
steps     678
Name: count, dtype: int64


In [32]:
import pandas as pd
import json

# Copy the original dataframe
df_clean = df.copy()

# Function to normalize JSON
def normalize_json(x):
    if isinstance(x, str):
        x = json.loads(x)  # parse JSON string
    if isinstance(x, list):
        return pd.DataFrame(x)  # explode list of dicts
    elif isinstance(x, dict):
        return pd.DataFrame([x])  # wrap dict in a DataFrame
    else:
        return pd.DataFrame()  # empty fallback

# Create a dictionary to hold separate tidy dfs by filename prefix
tidy_dfs = {}

# Loop through unique filename prefixes (everything before the first '_')
for prefix in df_clean['filename'].str.split('_').str[0].unique():
    df_subset = df_clean[df_clean['filename'].str.startswith(prefix)]
    temp_list = []
    
    for _, row in df_subset.iterrows():
        temp_df = normalize_json(row['raw_json'])
        if not temp_df.empty:
            temp_df['filename'] = row['filename']  # keep filename
            temp_list.append(temp_df)
    
    if temp_list:
        tidy_dfs[prefix] = pd.concat(temp_list, ignore_index=True)

# Now tidy_dfs contains:
# keys: 'body', 'heart', 'sleep', 'stress', 'steps'
# values: the corresponding tidy DataFrames
# Example: inspect the first few rows of body_battery
tidy_dfs.keys()


dict_keys(['body', 'heart', 'sleep', 'stress', 'steps'])

# body battery

In [45]:
# Access the body_battery dataframe
tidy_body = tidy_dfs['body']

# Show first 10 rows for inspection
tidy_body.head(10)

,filename,metric,date,charged,drained,startTimestampGMT,endTimestampGMT,startTimestampLocal,endTimestampLocal,bodyBatteryValuesArray,...,stressChartYAxisOrigin,stressValueDescriptorsDTOList,stressValuesArray,startGMT,endGMT,steps,pushes,primaryActivityLevel,activityLevelConstant,bodyBatteryValueDescriptorsDTOList
0,body_battery_2025-09-22.json,body,2025-09-22,45.0,71.0,2025-09-21T22:00:00.0,2025-09-22T22:00:00.0,2025-09-22T00:00:00.0,2025-09-23T00:00:00.0,"[[1758492000000, 38], [1758517200000, 81], [17...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,body_battery_2025-05-01.json,body,2025-05-01,36.0,30.0,2025-04-30T22:00:00.0,2025-05-01T22:00:00.0,2025-05-01T00:00:00.0,2025-05-02T00:00:00.0,"[[1746050400000, 5], [1746075420000, 35], [174...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,body_battery_2025-06-11.json,body,2025-06-11,80.0,74.0,2025-06-10T22:00:00.0,2025-06-11T22:00:00.0,2025-06-11T00:00:00.0,2025-06-12T00:00:00.0,"[[1749592800000, 9], [1749621240000, 85], [174...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,body_battery_2025-03-28.json,body,2025-03-28,52.0,50.0,2025-03-27T23:00:00.0,2025-03-28T23:00:00.0,2025-03-28T00:00:00.0,2025-03-29T00:00:00.0,"[[1743116400000, 14], [1743144660000, 60], [17...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
599,body_battery_2025-05-17.json,body,2025-05-17,45.0,42.0,2025-05-16T22:00:00.0,2025-05-17T22:00:00.0,2025-05-17T00:00:00.0,2025-05-18T00:00:00.0,"[[1747432800000, 5], [1747460520000, 47], [174...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
605,body_battery_2025-06-07.json,body,2025-06-07,53.0,53.0,2025-06-06T22:00:00.0,2025-06-07T22:00:00.0,2025-06-07T00:00:00.0,2025-06-08T00:00:00.0,"[[1749247200000, 5], [1749274920000, 58], [174...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,body_battery_2025-08-19.json,body,2025-08-19,72.0,71.0,2025-08-18T22:00:00.0,2025-08-19T22:00:00.0,2025-08-19T00:00:00.0,2025-08-20T00:00:00.0,"[[1755555660000, 17], [1755581760000, 79], [17...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
708,body_battery_2025-03-12.json,body,2025-03-12,37.0,79.0,2025-03-11T23:00:00.0,2025-03-12T23:00:00.0,2025-03-12T00:00:00.0,2025-03-13T00:00:00.0,"[[1741734000000, 47], [1741756500000, 84], [17...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
902,body_battery_2025-10-02.json,body,2025-10-02,54.0,67.0,2025-10-01T21:00:00.0,2025-10-02T21:00:00.0,2025-10-02T00:00:00.0,2025-10-03T00:00:00.0,"[[1759353660000, 27], [1759374720000, 79], [17...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1001,body_battery_2025-09-18.json,body,2025-09-18,89.0,66.0,2025-09-17T15:00:00.0,2025-09-18T15:00:00.0,2025-09-18T00:00:00.0,2025-09-19T00:00:00.0,"[[1758121200000, 5], [1758149640000, 88], [175...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
